## Predict picturebook lfw

The model made in VGGFace clasification lfw is based on a subset of 17 classes of the dataset 'Faces in the wild'. This model is read from disk and used to predict a small number of pictures collected from the internet. The pictures contain at least one image of the faces that were trained, but in several cases additional unknown faces are there. First Opencv is used to detect faces on the pictures. Then the faces are classified using the model and the results are evaluated. 

In [1]:
import numpy as np
import os
import time
#from keras.preprocessing import image
#from keras.applications.imagenet_utils import preprocess_input
#from keras.layers import Dense, Activation, Flatten
#from keras.layers import merge, Input
#from keras.models import Model
from keras.utils import np_utils
#from sklearn.utils import shuffle
#from sklearn.cross_validation import train_test_split
#from keras.preprocessing.image import load_img
from keras_vggface.vggface import VGGFace
from keras.models import model_from_json
import cv2
import glob
import PIL.Image
import numpy as np
import argparse

Using TensorFlow backend.


## Prepare Opencv

In [2]:
# Initialize the haarcascade classifier for face detection
cascade_file_src = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascade_file_src)

In [3]:
# Some command line arguments are needed to be able to execute the deep learning model for face detection
args = {
    "prototxt": "deploy.prototxt.txt",
    "model": "res10_300x300_ssd_iter_140000.caffemodel",
    }

In [4]:
# load the serialized model from disk
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe(args["prototxt"], args["model"])

[INFO] loading model...


## Load pictures and detect faces

We use the combined haarcascade and deep learning function in this case

In [6]:
def ext_faces3(image):
    # First haarcascade is used to find faces 
    # Using low values of scaleFactor and minNeighbours (with many false positives)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, 1.2, 3)
    facecrop = []
    # The faces are cropped and the resulting images are fed in a sequence to the neural net.
    for f in faces:
        x, y, w, h = [ v for v in f ]
        facecrop.append(image[y:y+h, x:x+w])
    confirmed = []
    for img in facecrop:    
        blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        # Probability function of 0.9 can be changed to filter false positives
        if detections[0, 0, 0, 2] > 0.9:
            face = cv2.resize(img,((224,224)))
            confirmed.append(face)
    print('# faces found:',len(confirmed))
    return(confirmed)
            

In [7]:
# iterate over directories and files
# detect faces using ext_faces
# make a list of preprocessed images 

import os

rootdir ='Picturebook/'
n = len(rootdir)
newdir="testoutput2"
img_data_list = []
path_list =[]
name_list = []
for subdir, dirs, files in os.walk(rootdir):
    crdir = newdir + subdir[n-1:]
    print(crdir)
    os.makedirs(crdir)
    for file in files:
        if file[-4:] in (".jpg",".JPG"):
            filepath = subdir + '/' + file
            print(filepath)
            image= cv2.imread(filepath)
            name = file[:-4]
            facecrop2 = ext_faces3(image)
            for i, face in enumerate(facecrop2):
                img_data_list.append(face)
                name_list.append(name + '-' + format(i))
                path_list.append(format(filepath))
                

testoutput2/
Picturebook//Ann Veneman.jpg
# faces found: 1
Picturebook//Halle-Berry-Intro.jpg
# faces found: 1
Picturebook//howard_dean.jpg
# faces found: 1
Picturebook//lavrov.jpg
# faces found: 2
Picturebook//Nicole_kidman.jpg
# faces found: 2
Picturebook//Richard Gephard.jpg
# faces found: 3
Picturebook//salma-hayek-ashley-judd-golden-globes.jpg
# faces found: 2
Picturebook//Cheney and Trump.jpg
# faces found: 1
Picturebook//Jeb and George Bush.jpg
# faces found: 2
Picturebook//Harisson Ford.jpg
# faces found: 2
Picturebook//Naomi_Watts_Liev_Schreiber.jpg
# faces found: 2
Picturebook//Meryl_Streep.jpg
# faces found: 1
Picturebook//Britney Spears.jpg
# faces found: 1
Picturebook//nancy-pelosi-accompanied.jpg
# faces found: 2
Picturebook//Jennifer-Garner-Baby2Baby-Gala-Pictures-Copy1.jpg
# faces found: 3
testoutput2/.ipynb_checkpoints


The values in the list above are the (maximum) sizes of the cropped faces detected. In almost every case these values are smaller then the input value of the model which is 224. The reason for this is these pictures are gathered from the internet and have a low resolution. The faces are resized to the right size (224x224) but as the complete resolution is not there in the original picture the classification might not use all features. In case of using your own picturebook this is less a problem as you probably have pictures with a higher resolution.

In [8]:
print(len(img_data_list),len(name_list))
name_list

26 26


['Ann Veneman-0',
 'Halle-Berry-Intro-0',
 'howard_dean-0',
 'lavrov-0',
 'lavrov-1',
 'Nicole_kidman-0',
 'Nicole_kidman-1',
 'Richard Gephard-0',
 'Richard Gephard-1',
 'Richard Gephard-2',
 'salma-hayek-ashley-judd-golden-globes-0',
 'salma-hayek-ashley-judd-golden-globes-1',
 'Cheney and Trump-0',
 'Jeb and George Bush-0',
 'Jeb and George Bush-1',
 'Harisson Ford-0',
 'Harisson Ford-1',
 'Naomi_Watts_Liev_Schreiber-0',
 'Naomi_Watts_Liev_Schreiber-1',
 'Meryl_Streep-0',
 'Britney Spears-0',
 'nancy-pelosi-accompanied-0',
 'nancy-pelosi-accompanied-1',
 'Jennifer-Garner-Baby2Baby-Gala-Pictures-Copy1-0',
 'Jennifer-Garner-Baby2Baby-Gala-Pictures-Copy1-1',
 'Jennifer-Garner-Baby2Baby-Gala-Pictures-Copy1-2']

The list above shows the names of the original pictures and the sequence of the number of faces collected from these

## Load model

In [9]:
# load json and create model
json_file = open('model_lfw.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_lfw.h5")
print("Loaded model from disk")


Loaded model from disk


In [10]:
loaded_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
__________

In [11]:
# Make array of list
img_data = np.array(img_data_list)
print (img_data.shape)

(26, 224, 224, 3)


In [12]:
num_of_samples = len(img_data_list)

In [13]:
# Make a prediction on the trained model
yhat = loaded_model.predict(img_data)
yhat.shape

(26, 17)

In [14]:
# The list of classes is loaded from file
classes = np.loadtxt("classes.csv",  dtype=str, delimiter=",")

In [15]:
classes

array(['Bill_Simon', 'Britney_Spears', 'Dick_Cheney', 'Halle_Berry',
       'Harrison_Ford', 'Howard_Dean', 'Jeb_Bush', 'Jennifer_Garner',
       'John_Snow', 'Meryl_Streep', 'Nancy_Pelosi', 'Naomi_Watts',
       'Nicole_Kidman', 'Richard_Gephardt', 'Salma_Hayek', 'Sergey_Lavrov',
       'Ann_Veneman'],
      dtype='<U16')

In [16]:
# The labels are assigned. By using a threshold (unknown_prob) pictures with a lower maximum probability can be assigned to a 
# separate category. This threshold is arbitrary and leads to false positives and false negatives.
unknown_prob = 0.5
results = []
for pred in yhat:
    if np.max(pred[::-1]) > unknown_prob:
        top_indices = pred.argsort()[-1:][::-1]
        result = [[str(classes[i]), pred[i]] for i in top_indices]
        result.sort(key=lambda x: x[1], reverse=True)
        results.append(result[0][0])
    else: results.append('unknown')

## Evaluation
In most cases the subject of the picture is classified right. But the other unknown faces on the pictures in various cases are incorrectly assigned to labels while they should be 'unknown'. The probability of these faces is too high for the threshold. Increasing the threshold might help but can decrease the number of right classified faces. 

In [17]:
len(results)
len(name_list)
for i, x in enumerate(results):
    print(name_list[i],results[i])

Ann Veneman-0 Ann_Veneman
Halle-Berry-Intro-0 Halle_Berry
howard_dean-0 Howard_Dean
lavrov-0 John_Snow
lavrov-1 unknown
Nicole_kidman-0 Britney_Spears
Nicole_kidman-1 Nicole_Kidman
Richard Gephard-0 Richard_Gephardt
Richard Gephard-1 unknown
Richard Gephard-2 Bill_Simon
salma-hayek-ashley-judd-golden-globes-0 Nicole_Kidman
salma-hayek-ashley-judd-golden-globes-1 Salma_Hayek
Cheney and Trump-0 Dick_Cheney
Jeb and George Bush-0 Sergey_Lavrov
Jeb and George Bush-1 Jeb_Bush
Harisson Ford-0 Harrison_Ford
Harisson Ford-1 Nicole_Kidman
Naomi_Watts_Liev_Schreiber-0 Naomi_Watts
Naomi_Watts_Liev_Schreiber-1 Howard_Dean
Meryl_Streep-0 Meryl_Streep
Britney Spears-0 Britney_Spears
nancy-pelosi-accompanied-0 Nancy_Pelosi
nancy-pelosi-accompanied-1 unknown
Jennifer-Garner-Baby2Baby-Gala-Pictures-Copy1-0 Jennifer_Garner
Jennifer-Garner-Baby2Baby-Gala-Pictures-Copy1-1 Naomi_Watts
Jennifer-Garner-Baby2Baby-Gala-Pictures-Copy1-2 unknown


In [22]:
# Generate an overview of the top 5 probabilities per picture
top=5
ereval = []
for pred in yhat:
    top_indices = pred.argsort()[-top:][::-1]
    erev = [[str(classes[i]), pred[i]] for i in top_indices]
    erev.sort(key=lambda x: x[1], reverse=True)
    ereval.append(erev)
# ereval

In case you want to inspect the faces captured from the pictures you can write them to folders below. These pictures can be used to be added  to the trainingset and retrain the model, with additional labels if you want. In the end the classification result can be used to generate a file of classified faces per picture that can be stored and used to search for faces in a collection of photographs/snapshots. 

In [18]:
for i, face in enumerate(img_data):
    if results[i] == 'unknown':
        cv2.imwrite("unknown/" + name_list[i] + "_" + str(i) + ".jpg", face)